## read from UDD

In [30]:
import psycopg2
import pandas as pd
# Replace these with your own database credentials
host1 = "udd-mirror.debian.net"
database1 = "udd"
user1 = "udd-mirror"
password1 = "udd-mirror"

# Establish the connection
connection_udd = psycopg2.connect(
    host=host1,
    database=database1,
    user=user1,
    password=password1
)

In [2]:
# Create a cursor object
cursor = connection_udd.cursor()
cursor.execute("SELECT * FROM packages where package='cmake'")
result = cursor.fetchall()
print(result)
cursor.close()
connection_udd.close()

[('cmake', '3.0.2-1+deb8u1', 'armel', 'Modestas Vainius <modax@debian.org>', 'Modestas Vainius', 'modax@debian.org', 'cross-platform, open-source make system', '47b53839da906127970f1e8c870afc2d', 'cmake', '3.0.2-1+deb8u1', None, 'cmake-data (= 3.0.2-1+deb8u1), procps, libarchive13, libc6 (>= 2.15), libcurl3 (>= 7.16.2), libexpat1 (>= 2.0.1), libgcc1 (>= 1:4.4.0), libstdc++6 (>= 4.9), zlib1g (>= 1:1.2.3.3)', 'gcc, make', 'codeblocks, eclipse, ninja-build', None, 'dpkg (>= 1.17.5~)', None, 10515, 'http://cmake.org/', 1946274, None, None, 'a311744036022be21b3b1f2950c00083c1ca4a46', None, 'devel', '064e4ddb427d20715459249ae3b0e4f8', None, 'optional', 'devel::buildtools, implemented-in::c++, interface::text-mode,\n role::program, scope::utility, uitoolkit::ncurses,\n works-with::software:source', None, None, None, None, None, '608ef14fc1974fa55b1f9bf27f069581fb525c3cc8acd01bdf74ebafbfc9bc4a', None, 'debian', 'jessie', 'main', 'foreign', None, 'pool/main/c/cmake/cmake_3.0.2-1+deb8u1_armel.de

In [5]:
query = "SELECT * FROM packages where package='cmake'"
df = pd.read_sql_query(query, connection_udd)
connection_udd.close()

/var/folders/hs/yzhw94b12617bzv5gygdhc780000gn/T/ipykernel_13691/1390185480.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [6]:
df

,package,version,architecture,maintainer,maintainer_name,maintainer_email,description,description_md5,source,source_version,...,provides,conflicts,sha256,original_maintainer,distribution,release,component,multi_arch,package_type,filename
0,cmake,3.0.2-1+deb8u1,armel,Modestas Vainius <modax@debian.org>,Modestas Vainius,modax@debian.org,"cross-platform, open-source make system",47b53839da906127970f1e8c870afc2d,cmake,3.0.2-1+deb8u1,...,None,None,608ef14fc1974fa55b1f9bf27f069581fb525c3cc8acd0...,None,debian,jessie,main,foreign,None,pool/main/c/cmake/cmake_3.0.2-1+deb8u1_armel.deb
1,cmake,3.0.2-1+deb8u1,armhf,Modestas Vainius <modax@debian.org>,Modestas Vainius,modax@debian.org,"cross-platform, open-source make system",47b53839da906127970f1e8c870afc2d,cmake,3.0.2-1+deb8u1,...,None,None,25b0af094089e6d1b0ee31fe67a0d75aff6bf5e8031ecf...,None,debian,jessie,main,foreign,None,pool/main/c/cmake/cmake_3.0.2-1+deb8u1_armhf.deb
2,cmake,3.0.2-1+deb8u1,i386,Modestas Vainius <modax@debian.org>,Modestas Vainius,modax@debian.org,"cross-platform, open-source make system",47b53839da906127970f1e8c870afc2d,cmake,3.0.2-1+deb8u1,...,None,None,32d3f0d779ba57e3f90f10db75566286663252422db65d...,None,debian,jessie,main,foreign,None,pool/main/c/cmake/cmake_3.0.2-1+deb8u1_i386.deb
3,cmake,3.0.2-1+deb8u1,amd64,Modestas Vainius <modax@debian.org>,Modestas Vainius,modax@debian.org,"cross-platform, open-source make system",47b53839da906127970f1e8c870afc2d,cmake,3.0.2-1+deb8u1,...,None,None,d0a2439f6e2e113f9357e1cb902c679688fc48ec4b3bbf...,None,debian,jessie,main,foreign,None,pool/main/c/cmake/cmake_3.0.2-1+deb8u1_amd64.deb
4,cmake,3.7.2-1,s390x,Debian CMake Team <pkg-cmake-team@lists.alioth...,Debian CMake Team,pkg-cmake-team@lists.alioth.debian.org,"cross-platform, open-source make system",47b53839da906127970f1e8c870afc2d,cmake,3.7.2-1,...,None,None,eb3722f9cc190e4633760ba4588381d021b1184c8be2b0...,None,debian,stretch,main,foreign,None,pool/main/c/cmake/cmake_3.7.2-1_s390x.deb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,cmake,3.27.5-1,i386,Debian CMake Team <pkg-cmake-team@lists.alioth...,Debian CMake Team,pkg-cmake-team@lists.alioth.debian.org,"cross-platform, open-source make system",b60d0479b91d22665e465b5cec287c83,cmake,3.27.5-1,...,None,None,9d1776efbca77769852d6c34ace6cc222197ef4bc4d7c9...,None,debian,sid,main,foreign,None,pool/main/c/cmake/cmake_3.27.5-1_i386.deb
93,cmake,3.27.5-1,amd64,Debian CMake Team <pkg-cmake-team@lists.alioth...,Debian CMake Team,pkg-cmake-team@lists.alioth.debian.org,"cross-platform, open-source make system",b60d0479b91d22665e465b5cec287c83,cmake,3.27.5-1,...,None,None,b567932f02c9285528d236bd63bfb5bc94e868846b3304...,None,debian,sid,main,foreign,None,pool/main/c/cmake/cmake_3.27.5-1_amd64.deb
94,cmake,3.27.5-1,ppc64el,Debian CMake Team <pkg-cmake-team@lists.alioth...,Debian CMake Team,pkg-cmake-team@lists.alioth.debian.org,"cross-platform, open-source make system",b60d0479b91d22665e465b5cec287c83,cmake,3.27.5-1,...,None,None,059e1ba75f375554edcf1ae02839ee9468fb63c35e74fc...,None,debian,sid,main,foreign,None,pool/main/c/cmake/cmake_3.27.5-1_ppc64el.deb
95,cmake,3.27.5-1,riscv64,Debian CMake Team <pkg-cmake-team@lists.alioth...,Debian CMake Team,pkg-cmake-team@lists.alioth.debian.org,"cross-platform, open-source make system",b60d0479b91d22665e465b5cec287c83,cmake,3.27.5-1,...,None,None,28e2b56c192382f9a290067044fa4b637092de8231b20e...,None,debian,sid,main,foreign,None,pool/main/c/cmake/cmake_3.27.5-1_riscv64.deb


## webcrawl

In [27]:
import requests
from bs4 import BeautifulSoup

# Define the URL you want to crawl
url = 'https://snapshot.debian.org/binary/cmake/'

# Send an HTTP GET request to the URL
response = requests.get(url)

# Initialize an empty list to store the extracted text
text_list = []

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all the <ul> elements
    ul_elements = soup.find_all('ul')

    # Loop through the <ul> elements to find <li> and <a> elements
    for ul in ul_elements:
        # Find all the <li> elements within the current <ul>
        li_elements = ul.find_all('li')
        
        # Loop through the <li> elements to find <a> elements
        for li in li_elements:
            # Find all the <a> elements within the current <li>
            a_elements = li.find_all('a')
            
            # Loop through the <a> elements to extract and append their text to the list
            for a in a_elements:
                text = a.get_text()
                text_list.append(text)


In [36]:
text_list

['snapshot.debian.org',
 'c*',
 '3.27.5-1 (source: cmake 3.27.5-1)',
 '3.27.4-1 (source: cmake 3.27.4-1)',
 '3.27.4-1~bpo12+1 (source: cmake 3.27.4-1~bpo12+1)',
 '3.27.3-1 (source: cmake 3.27.3-1)',
 '3.27.2-1 (source: cmake 3.27.2-1)',
 '3.27.2-1~bpo12+1 (source: cmake 3.27.2-1~bpo12+1)',
 '3.27.1-2+b1 (source: cmake 3.27.1-2)',
 '3.27.1-2 (source: cmake 3.27.1-2)',
 '3.27.1-1 (source: cmake 3.27.1-1)',
 '3.27.0-2 (source: cmake 3.27.0-2)',
 '3.27.0-1 (source: cmake 3.27.0-1)',
 '3.27.0~rc5-1 (source: cmake 3.27.0~rc5-1)',
 '3.27.0~rc4-3 (source: cmake 3.27.0~rc4-3)',
 '3.27.0~rc4-2 (source: cmake 3.27.0~rc4-2)',
 '3.27.0~rc4-1 (source: cmake 3.27.0~rc4-1)',
 '3.26.4-4 (source: cmake 3.26.4-4)',
 '3.26.4-4~bpo12+1 (source: cmake 3.26.4-4~bpo12+1)',
 '3.26.4-3 (source: cmake 3.26.4-3)',
 '3.26.4-2 (source: cmake 3.26.4-2)',
 '3.26.4-1 (source: cmake 3.26.4-1)',
 '3.25.1-1 (source: cmake 3.25.1-1)',
 '3.25.1-1~bpo11+1 (source: cmake 3.25.1-1~bpo11+1)',
 '3.25.0-3 (source: cmake 3.25.0-3

In [40]:
import re

binary_pkg_v_map={}

for text in text_list:
    # Define a regular expression pattern to match 'XXX' and 'ZZZ@WWW'
    # pattern = r'/(\b[^()]+\b)\s\(source:\s*[^()]+\s([^()]+)\)/gm'
    pattern = r'(\S+)\s\(source:\s*[^()]+\s(\S+)\)'

    # Use re.search to find the pattern in the text
    match = re.search(pattern, text)

    # Check if a match is found
    if match:
        binary_v = match.group(1)  
        pkg_v = match.group(2)  
        binary_pkg_v_map[binary_v]=pkg_v

df1 = pd.DataFrame(binary_pkg_v_map.items(), columns=['binary_v', 'package_v'])

In [42]:
df1

,binary_v,package_v
0,3.27.5-1,3.27.5-1
1,3.27.4-1,3.27.4-1
2,3.27.4-1~bpo12+1,3.27.4-1~bpo12+1
3,3.27.3-1,3.27.3-1
4,3.27.2-1,3.27.2-1
...,...,...
260,2.0.6-2,2.0.6-2
261,2.0.6-1,2.0.6-1
262,2.0.5-1,2.0.5-1
263,2.0.3-1,2.0.3-1


In [43]:
query = "SELECT version, date FROM upload_history where source='cmake'"
df2 = pd.read_sql_query(query, connection_udd)
# connection_udd.close()

/var/folders/hs/yzhw94b12617bzv5gygdhc780000gn/T/ipykernel_13958/1965905114.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql_query(query, connection_udd)


In [44]:
df2

,version,date
0,0.96-1,2001-12-04 19:57:16+00:00
1,0.98-1,2001-12-19 19:56:35+00:00
2,1.0-1,2002-01-11 20:01:00+00:00
3,1.2-1,2002-01-17 19:55:23+00:00
4,1.2-2,2002-02-15 19:57:31+00:00
...,...,...
190,3.27.1-2,2023-07-29 22:21:08+00:00
191,3.27.2-1,2023-08-11 10:50:02+00:00
192,3.27.3-1,2023-08-21 09:07:50+00:00
193,3.27.4-1,2023-08-26 18:20:56+00:00


In [46]:
merged_df = df1.merge(df2, left_on='package_v', right_on='version', how='outer', indicator=True)  # Use 'how' parameter to specify the type of join (inner, outer, left, right)

In [52]:
merged_df[merged_df['_merge']=='both']

,binary_v,package_v,version,date,_merge
0,3.27.5-1,3.27.5-1,3.27.5-1,2023-09-15 12:20:54+00:00,both
1,3.27.4-1,3.27.4-1,3.27.4-1,2023-08-26 18:20:56+00:00,both
2,3.27.4-1~bpo12+1,3.27.4-1~bpo12+1,NaN,NaT,left_only
3,3.27.3-1,3.27.3-1,3.27.3-1,2023-08-21 09:07:50+00:00,both
4,3.27.2-1,3.27.2-1,3.27.2-1,2023-08-11 10:50:02+00:00,both
...,...,...,...,...,...
275,NaN,NaN,1.4.7-1,2003-01-07 00:47:12+00:00,right_only
276,NaN,NaN,1.6.4-1,2003-02-15 17:17:35+00:00,right_only
277,NaN,NaN,1.6.5-1,2003-02-24 17:47:29+00:00,right_only
278,NaN,NaN,1.8.2.1-1,2003-12-06 23:02:07+00:00,right_only


## verify with cvetriage

In [1]:
import os
import psycopg2
import pandas as pd
from dotenv import load_dotenv
load_dotenv('../pipeline_monitoring/.env')

# Replace these with your own database credentials
host2 = os.environ.get('CVETRIAGE_POSTGRES_HOSTNAME')
database2 = os.environ.get('CVETRIAGE_POSTGRES_DB')
user2 = os.environ.get('CVETRIAGE_POSTGRES_USERNAME')
password2 = os.environ.get('CVETRIAGE_POSTGRES_PASSWORD')

# Establish the connection
connection_cve = psycopg2.connect(
    host=host2,
    database=database2,
    user=user2,
    password=password2
)

In [63]:
binary_name='cmake'
query = f'''SELECT slv.version_number, slv.release_date from scantist_library sl
join scantist_library_version slv on slv.library_id=sl.id
where sl.name='{binary_name}'
and sl.platform='Debian'
and slv.is_valid=True
order by slv.release_date desc
'''
df3 = pd.read_sql_query(query, connection_cve)
# connection_udd.close()

/var/folders/hs/yzhw94b12617bzv5gygdhc780000gn/T/ipykernel_13958/1598810402.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df3 = pd.read_sql_query(query, connection_cve)


In [64]:
df3

,version_number,release_date
0,3.26.4-4~bpo12+1,2023-07-28 23:34:23+00:00
1,3.18.4-2+deb11u1~bpo10+1,2021-12-18 04:03:26+00:00
2,3.22.1-1~bpo11+1,2021-12-10 01:00:20+00:00
3,3.22.1-1,2021-12-07 16:19:44+00:00
4,3.22.0-1~bpo11+1,2021-11-22 05:36:59+00:00
...,...,...
180,1.2-2,2002-02-14 02:36:25+00:00
181,1.0-1,2002-01-09 01:20:12+00:00
182,1.2-1,2002-01-09 01:20:12+00:00
183,0.98-1,2001-12-17 03:54:32+00:00


In [78]:
merged_df2 = df1.merge(df3, left_on='binary_v', right_on='version_number', how='outer', indicator=True)  # Use 'how' parameter to specify the type of join (inner, outer, left, right)

In [81]:
merged_df2[merged_df2['_merge']=='right_only']

,binary_v,package_v,version_number,release_date,_merge
265,NaN,NaN,2.0.2-1,2004-06-22 21:51:38+00:00,right_only
266,NaN,NaN,2.0.1-1,2004-06-07 06:08:50+00:00,right_only
267,NaN,NaN,1.8.2.1-1,2003-11-26 05:46:54+00:00,right_only
268,NaN,NaN,1.8.1-1,2003-11-08 17:43:02+00:00,right_only
269,NaN,NaN,1.6.7-2,2003-06-22 21:39:03+00:00,right_only
270,NaN,NaN,1.6.7-1,2003-06-22 21:06:46+00:00,right_only
271,NaN,NaN,1.6.5-1,2003-02-23 19:25:58+00:00,right_only
272,NaN,NaN,1.6.4-1,2003-02-14 00:06:12+00:00,right_only
273,NaN,NaN,1.4.7-1,2003-01-06 03:02:36+00:00,right_only
274,NaN,NaN,1.4.6-1,2002-10-24 05:19:47+00:00,right_only


In [76]:
df1[df1['binary_v'].str.startswith('3.26.4')]

,binary_v,package_v
15,3.26.4-4,3.26.4-4
16,3.26.4-4~bpo12+1,3.26.4-4~bpo12+1
17,3.26.4-3,3.26.4-3
18,3.26.4-2,3.26.4-2
19,3.26.4-1,3.26.4-1
